In [1]:
import psycopg2
import pandas as pd
pd.options.mode.chained_assignment = None
from tqdm.notebook import tqdm
import math

In [2]:
# Loading data
conn = psycopg2.connect(database="full_STUPS",
                        user="postgres",
                        host='localhost',
                        password="postgres",
                        port=5432)
cur = conn.cursor()
cur.execute('SELECT id_composition as "e1", id_composition_lien as "e2", 2147483647 as "id_lot" FROM lot_echantillon where id_composition is not null and id_composition_lien is not null')
samples = cur.fetchall()
conn.close()
colnames = [desc[0] for desc in cur.description]

df = pd.DataFrame(list(samples), columns=colnames)
#df = df.iloc[:500, :]
display(df)

,e1,e2,id_lot
0,13080,13077,2147483647
1,17649,17648,2147483647
2,23027,22296,2147483647
3,28413,28415,2147483647
4,35824,35748,2147483647
...,...,...,...
3178,58648,58778,2147483647
3179,58648,59072,2147483647
3180,59392,2022,2147483647
3181,59390,59385,2147483647


In [3]:
def in_df(name_1, v1, name_2, v2, df):
    for i in df.index:
        if df[name_1][i] == v1 and df[name_2][i] == v2:
            return True
    return False

In [4]:
def undirect_links(df):
    print("Undirecting links.")
    for i in tqdm(df.index):
        df.loc[df.shape[0]] = [df['e2'][i], df['e1'][i], df['id_lot'][i]]
    df = df.drop_duplicates(keep='first', ignore_index=True)
    return df

def expand(df, start_search_index):
    df_2 = df.copy()
    for i in tqdm(df.index):
        neighbors = []
        e1 = df['e1'][i]
        e2 = df['e2'][i]
        id_lot = df['id_lot'][i]
        for j in range(start_search_index, len(df.index)):
            e1_2 = df['e1'][j]
            e2_2 = df['e2'][j]

            if e1 != e2_2 and e2 == e1_2:
                neighbors.append(e2_2)

        if len(neighbors) != 0:
            for n in neighbors:
                if e1 != n:
                    df_2.loc[len(df_2)] = [e1, n, id_lot]
                    df_2.loc[len(df_2)] = [n, e1, id_lot]

    df_2 = df_2.drop_duplicates(keep='first', ignore_index=True)
    new_nodes = df_2.shape[0] - df.shape[0]
    print(f'New nodes : {new_nodes}')
    if new_nodes > 0:
        return expand(df_2, len(df.index))
    else:
        return df_2

def complete_batch_v2(df):
    df = undirect_links(df)
    print(f'Dataframe new shape : {df.shape}')
    df = expand(df, 0)
    df = undirect_links(df)
    print(f'Dataframe new shape : {df.shape}')
    return df

df = complete_batch_v2(df)
#print(df)

Undirecting links.


  0%|          | 0/3183 [00:00<?, ?it/s]

Dataframe new shape : (6291, 3)


  0%|          | 0/6291 [00:00<?, ?it/s]

New nodes : 3532


...

New nodes : 2


  0%|          | 0/11323 [00:00<?, ?it/s]

New nodes : 0
Undirecting links.


  0%|          | 0/11323 [00:00<?, ?it/s]

Dataframe new shape : (11323, 3)


In [5]:
i = 0

for ind in tqdm(df.index):
    e1 = df['e1'][ind]
    e2 = df['e2'][ind]
    id_lot = min(i, df['id_lot'][ind])
    
    if id_lot == i:
        i += 1
        df['id_lot'][ind] = id_lot
        
    for ind2 in df.index:
        e1_2 = df['e1'][ind2]
        e2_2 = df['e2'][ind2]
        id_lot_2 = df['id_lot'][ind2]

        if e1_2 == e1 or e1_2 == e2 or e2_2 == e1 or e2_2 == e2:
            df['id_lot'].at[ind2] = id_lot

  0%|          | 0/11323 [00:00<?, ?it/s]

In [6]:
display(df)

,e1,e2,id_lot
0,13080,13077,0
1,17649,17648,1
2,23027,22296,2
3,28413,28415,3
4,35824,35748,4
...,...,...,...
11318,16662,16654,344
11319,16661,16653,344
11320,16653,16661,344
11321,16653,16662,344


In [8]:
conn = psycopg2.connect(database="full_STUPS",
                        user="postgres",
                        host='localhost',
                        password="postgres",
                        port=5432)

for ind in tqdm(df.index):
    cur = conn.cursor()
    if not math.isnan(df["e1"][ind]) and not math.isnan(df["e2"][ind]):
        cur.execute(f'INSERT INTO lot_complet VALUES ({df["e1"][ind]}, {df["e2"][ind]}, {df["id_lot"][ind]})')

conn.commit()
conn.close()

  0%|          | 0/11323 [00:00<?, ?it/s]